In [24]:
# import SharedArray as sa
# sa.delete("shm://{}.grasp_img".format('indy7'))
# sa.delete("shm://{}.arm_img".format('indy7'))
# sa.delete("shm://{}.rh_vals".format('indy7'))
# sa.delete("shm://{}.result".format('indy7'))
# sa.delete("shm://{}.query_in".format('indy7'))
# sa.delete("shm://{}.response_out".format('indy7'))
# sa.delete("shm://{}.query_quit".format('indy7'))

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "filters")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = True
CLEARANCE = 1e-3

ROBOT_TYPE = RobotType.indy7
ROBOT_NAME = "indy0"
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]
TOOL_XYZ = (0,0,0.14)
TOOL_RPY = (-np.pi/2,0,0)
GRIP_DEPTH = 0.05
HOME_POSE = (0,0,0,0,0,0)

# ROBOT_TYPE = RobotType.panda
# ROBOT_NAME = "panda0"
# TOOL_LINK = "panda0_hand"
# TOOL_LINK_BUNDLE = ["panda0_hand", "panda0_link6"]
# TOOL_XYZ = (0,0,0.112)
# TOOL_RPY = (-np.pi/2,0,0)
# GRIP_DEPTH = 0.03
# HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)


assert TOOL_LINK_BUNDLE[0] == TOOL_LINK, "TOOL_LINK_BUNDLE should be in reverse order including actor's link as the first item"

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection_list
[False]


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", rname=ROBOT_NAME, _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
mplan = MoveitPlanner(pscene)
gcheck = GraspChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
lcheck = LatticedChecker(pscene, gcheck)
checkers_all = [gcheck, rcheck, lcheck]

In [12]:
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

## ui

In [13]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


# Object Classes

In [14]:
from pkg.utils.gjk import get_point_list, get_gjk_distance
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

##
# @class ObstacleBase
# @brief base class for obstacle generators
class ObstacleBase:
    RTH_MIN = None ## R: center ~ nearest point
    RTH_MAX = None
    RPY_MIN = None
    RPY_MAX = None
    DIM_MIN = None
    DIM_MAX = None
    GTYPE = None
    COLOR = (0.7,0.7,0.7,1)
    
    def __init__(self, gscene, name, sampler=np.random.uniform, DIM=None, RTH=None, RPY=None):
        self.name = name
        self.DIM = sampler(self.DIM_MIN, self.DIM_MAX) if DIM is None else DIM
        self.RTH = sampler(self.RTH_MIN, self.RTH_MAX) if RTH is None else RTH
        self.RPY = sampler(self.RPY_MIN, self.RPY_MAX) if RPY is None else RPY
        self.RPY[2] += self.RTH[1]
        self.XYZ = np.array(cyl2cart(*self.RTH))
        verts_rotated = np.matmul(Rot_rpy(self.RPY), (DEFAULT_VERT_DICT[self.GTYPE]*self.DIM).transpose())
        xy_normed = self.XYZ[:2]/(np.linalg.norm(self.XYZ[:2])+1e-6)
        verts_r_compo = np.dot(xy_normed, verts_rotated[:2,:])
        self.XYZ[:2] -= xy_normed[:2]*np.min(verts_r_compo)
        self.RTH[0] -= np.min(verts_r_compo)
        self.geometry = gscene.create_safe(gtype=self.GTYPE, name=self.name, link_name="base_link", 
                                  dims=self.DIM, center=tuple(self.XYZ), rpy=self.RPY, 
                                  color=self.COLOR, display=True, collision=True, fixed=True)
        self.subgeo_list = []
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_me, raddii_me = self.geometry.get_vertice_radius()
        verts_me_global = np.add(np.matmul(verts_me, self.geometry.orientation_mat.transpose()), 
                                 self.geometry.center)
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_me_global))-radii-raddii_me < 1e-4
        
##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,1)
    
    def __init__(self, gscene, name, floor_height=None, *args, **kwargs):
        assert floor_height is not None, "floor_height needed"
        if floor_height > self.RTH_MIN[2]:
            self.RTH_MIN = self.RTH_MIN[:2]+(floor_height,)
        self.H = 0.4
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
            
    
##
# @class Box
# @brief box with the top and the front side open
class Box(WorkPlane):
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    COLOR =  (0.8,0.8,0.2,0.5)
    H_RANGE = (0.3, 0.6)
    THICKNESS = 0.05
    def __init__(self, gscene, name, H=None, **kwargs):
        WorkPlane.__init__(self, gscene=gscene, name=name, **kwargs)
        self.H = np.random.uniform(*self.H_RANGE) if H is None else H

        ## back wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_bw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(self.DIM[0]/2+self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## left wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_lw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, -self.DIM[1]/2-self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        ## right wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_rw", link_name="base_link", 
            dims=(self.DIM[0], self.THICKNESS, self.H), center=(0, self.DIM[1]/2+self.THICKNESS/2, self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

        
##
# @class SideBox
# @brief box with a side face open
class SideBox(Box):
    RTH_MIN = (0.3, -np.pi/2, -0.1)
    RTH_MAX = (0.5, +np.pi/2, +0.4)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.4, 0.6, 0.05)
    DIM_MAX = (0.4, 0.6, 0.05)
    COLOR =  (0.2,0.2,0.8,1)
    H_RANGE = (0.4, 0.4)
    THICKNESS = 0.05
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## top
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_tp", link_name="base_link", 
            dims=(self.DIM[0], self.DIM[1], self.THICKNESS), center=(0, 0, self.H+self.THICKNESS/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))

##
# @class TopBox
# @brief box with the top face open
class TopBox(Box):
    DIM_MIN = (0.3, 0.3, 0.05)
    DIM_MAX = (0.6, 0.6, 0.05)
    H_RANGE = (0.3, 0.6)
    def __init__(self, gscene, name, **kwargs):
        Box.__init__(self, gscene=gscene, name=name, **kwargs)

        ## front wall
        self.subgeo_list.append(gscene.create_safe(
            gtype=self.GTYPE, name=self.name+"_fw", link_name="base_link", 
            dims=(self.THICKNESS, self.DIM[1], self.H), center=(-self.DIM[0]/2-self.THICKNESS/2, 0,self.H/2), rpy=(0,0,0),
            color=self.COLOR, display=True, collision=True, fixed=True,
            parent=self.name))
        
##
# @class Floor
# @brief Floor - lowerbound of the workspace
class Floor(ObstacleBase):
    RTH_MIN = (0.0, 0, -0.5)
    RTH_MAX = (0.0, 0, -0.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    
##
# @class Ceiling
# @brief Ceiling - upperbound of the workspace
class Ceiling(ObstacleBase):
    RTH_MIN = (0.0, 0, 1.0)
    RTH_MAX = (0.0, 0, 2)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (3, 3, 0.1)
    DIM_MAX = (3, 3, 0.1)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Wall
# @brief define horizontal boundary of the workspace
class Wall(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (2.0, np.pi, 0)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.1, 6, 3)
    DIM_MAX = (0.1, 6, 3)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.7,0.7,0.7,0.5)
    
##
# @class Pole
# @brief occasional poles that obstruct robot motion
class Pole(ObstacleBase):
    RTH_MIN = (0.3, -np.pi, 0)
    RTH_MAX = (0.3, +np.pi, 0)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.1, 0.1, 4)
    DIM_MAX = (0.1, 0.1, 4)
    GTYPE = GEOTYPE.BOX
    COLOR = (0.7,0.7,0.7,0.3)
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, -np.pi)
    RPY_MAX = (0, 0, +np.pi)
    DIM_MIN = (0.05, 0.1, 0.2)
    DIM_MAX = (0.05, 0.1, 0.2)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.8,0.2,1)
    def __init__(self, gscene, name, workplane, XYZ_LOC=None, **kwargs):
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.geometry.set_offset_tf(center = self.XYZ)
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
        
        
def clear_class(gscene, key, Nmax):
    for iw in range(Nmax):
        gname = "{}_{}".format(key, iw)
        if gname in gscene.NAME_DICT:
            gscene.remove(gscene.NAME_DICT[gname])

            
def redistribute_class(gscene, obstacle_class, key, Nmax, workplane_avoid=None):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    for iw in range(np.random.choice(Nmax)):
        obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        while workplane_avoid is not None and workplane_avoid.is_overlapped_with(obs.geometry):
            obs = obstacle_class(gscene, "{}_{}".format(key, iw))
        obs_list.append(obs)
    return obs_list

            
def disperse_objects(gscene, object_class, key, Nmax, workplane_on):
    clear_class(gscene, key, Nmax)
        
    obs_list = []
    while len(obs_list)<Nmax:
        iw = len(obs_list)
        obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on)
        remove_this = False
        for obs_pre in obs_list:
            if obs_pre.is_overlapped_with(obs.geometry):
                remove_this = True
                break
        if remove_this:
            gscene.remove(obs.geometry)
        else:
            obs_list.append(obs)
    return obs_list


def add_object(pscene, obj, HANDLE_THICKNESS=1e-6, HANDLE_COLOR = (1,0,0,0)):
    gscene = pscene.gscene
    handles = []
    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_tp_a", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_tp_b", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bt_a", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,-obj.DIM[2]/2+GRIP_DEPTH/2), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bt_b", link_name="base_link", 
                       dims=(GRIP_DEPTH, obj.DIM[1],HANDLE_THICKNESS), center=(0,0,-obj.DIM[2]/2+GRIP_DEPTH/2), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_ft_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_ft_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bk_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name=obj.name+"_hdl_bk_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    action_points_dict = {"placement": PlacePoint("placement", obj.geometry, [0,0,-obj.DIM[2]/2-CLEARANCE], [0,0,0])}
    action_points_dict.update({handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles})
    obj_pscene = pscene.create_subject(oname=obj.name, gname=obj.name, _type=CustomObject, 
                                 action_points_dict=action_points_dict)
    return obj_pscene, handles

   Use a production WSGI server instead.
 * Debug mode: off


In [15]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/filters/indy7/20210225-035030



### sampling 3-obj

In [16]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

results_dict_3_shelf = {0:[], 1:[], 2:[], 3:[]}
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}

In [17]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    mplan.update_gscene()
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)

    ppline.set_motion(mplan)
    ppline.set_sampler(tplan)

    gtimer = GlobalTimer.instance()
    gtimer.reset()



    for checker_idx in range(4):
        mplan.motion_filters = checkers_all[:checker_idx]
        goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
        mplan.reset_log(True)
        gtimer.tic("plan")
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
        elapsed = gtimer.toc("plan")/1000
        schedules = ppline.find_schedules()
        res = len(schedules)>0
        if res:
            schedule = ppline.sort_schedule(schedules)[0]
            move_num = len(schedule)-1
        else:
            move_num = 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_shelf[checker_idx].append([elapsed, move_num, plan_num, fail_num, res])

node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp

node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = success
branching: 2->5 (29.95/600.0 s, steps/err: 49(4587.9638195 ms)/0.0016246374715)
======================= terminated 0: first answer acquired ===============================
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail


node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 3->5 (0.58/600.0 s, steps/err: 52(136.054039001 ms)/0.00153264356043)
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'gr

node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 2->13 (25.13/600.0 s, steps/err: 3(60.8808994293 ms

node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 6->23 (31.26/600.0 s, steps/err: 19(51.1059761047 ms)/0.00121232414855)
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = success
branching: 11->24 (31.4/600.0 s, steps/err: 53(130.364179611 ms)/0.00162731353954)
node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('wp', 'gp', 'wp') = fail
nod

...

node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->2 (35.27/600.0 s, steps/err: 26(40.745973587 ms)/0.000903764691497)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->3 (35.36/600.0 s, steps/err: 33(94.0980911255 ms)/0.00182103416311)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 2->4 (65.59/600.0 s, steps/err: 19(19.8769569397 ms)/0.00206219310777)
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp

node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 5->13 (528.96/600.0 s, steps/err: 38(47.6450920105 ms)/0.000864972198906)
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 5->14 (564.28/600.

node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = success
branching: 1->6 (10.78/600.0 s, steps/err: 47(75.6039619446 ms)/0.00194422080998)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'g

node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = success
branching: 14->17 (39.22/600.0 s, steps/err: 28(49.9660968781 ms)/0.00145839938937)
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = success
branching: 5->18 (39.26/600.0 s, steps/err: 25(32.8009128571 ms)/0.00174948068057)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 5->19 (39.32/600.0 s, steps/err: 29(52.9968738556 ms)/0.00140053939501)
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'wp')->

node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->1 (0.11/600.0 s, steps/err: 19(98.8600254059 ms)/0.00146010469369)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->2 (0.16/600.0 s, steps/err: 27(49.9970912933 ms)/0.00141126330777)
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->3 (0.23/600.0 s, steps/err: 13(50.971031189 ms)/0.0017261964906)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp',

In [18]:
for k,v in results_dict_3_shelf.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

0: [231.14573168754578, 2.2857142857142856, 52.57142857142857, 45.714285714285715, 3, 10]
1: [56.56532707214355, 3.6, 30.7, 10.3, 0, 10]
2: [15.33657410144806, 3.0, 15.6, 2.5, 0, 10]
3: [7.917086791992188, 3.6, 13.4, 1.2, 0, 10]


In [19]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_shelf.json"), results_dict_3_shelf)

In [20]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/filters/indy7/20210225-050109



### sampling 3-obj

In [21]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

results_dict_3_shelf = {0:[], 1:[], 2:[], 3:[]}
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}

In [22]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    mplan.update_gscene()
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)

    ppline.set_motion(mplan)
    ppline.set_sampler(tplan)

    gtimer = GlobalTimer.instance()
    gtimer.reset()



    for checker_idx in range(4):
        mplan.motion_filters = checkers_all[:checker_idx]
        goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
        mplan.reset_log(True)
        gtimer.tic("plan")
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
        elapsed = gtimer.toc("plan")/1000
        schedules = ppline.find_schedules()
        res = len(schedules)>0
        if res:
            schedule = ppline.sort_schedule(schedules)[0]
            move_num = len(schedule)-1
        else:
            move_num = 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_shelf[checker_idx].append([elapsed, move_num, plan_num, fail_num, res])

node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = success
branching: 0->1 (10.06/600.0 s, steps/err: 25(25.0520706177 ms)/0.00228451232925)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 1->2 (35.25/600.0 s, steps/err: 6(21.8179225922 ms)/0.00216195254508)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 1->3 (35.27/600.0 s, steps/err: 16(23.4870910645 ms)/0.00205472227058)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 2->4 (40.35/600.0 s, steps/err: 25(34.3968868256 ms)/0.00150432558553)
n

node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 6->15 (423.2/600.0 s, steps/err: 62(38.134098053 ms)/0.00177396420015)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 1->16 (428.32/600.0 s, steps/err: 37(67.2070980072 ms)/0.00152048026882)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 1->17 (458.53/600.0 s, steps/err: 30(44.7208881378 ms)/0.000988902192742)
node: ('wp', 'wp', 'wp')-

node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 1->4 (40.6/600.0 s, steps/err: 16(29.8039913177 ms)/0.00133467826583)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 4->5 (45.64/600.0 s, steps/err: 16(27.9951095581 ms)/0.00158343206712)
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: 

node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'wp') = success
branching: 13->14 (91.7/600.0 s, steps/err: 72(62.9448890686 ms)/0.00156621596489)
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = success
branching: 14->15 (92.01/600.0 s, steps/err: 66(307.075023651 ms)/0.00152902321028)
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'grip0', 'gp')->('wp', 'wp', 'gp') = success
branching: 12->16 (92.04/600.0 s, steps/err: 27(26.9551277161 ms)/0.00179120138874)
node: ('wp', 'grip0', 'gp')->('wp', 'wp', 'gp') = success
branching: 12->17 (92.06/600.0 s, steps/err: 7(20.9050178528 ms)/0.00150325794958)
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') =

node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('gp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 8->31 (134.44/600.0 s, steps/err: 41(44.7850227356 ms)/0.00167468250613)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('grip0', 'gp', 'gp')->('gp', 'gp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('gp', 'gp', 'wp')->('gp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
node: ('wp', 'gp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('gp', 'gp', 'wp')->('gp', 

...

node: ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'gp')->('wp', 'wp', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('gp', 'gp', 'grip0')->('gp', 'gp', 'gp') = fail
node: ('gp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('gp', 'grip0', 'gp')->('gp', 'gp', 'gp') = fail
node: ('wp', 'gp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node: ('gp', 'wp', 'gp')->('gp', 'wp', 'grip0') = fail
node: ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp') = fail
node: ('gp', 'gp', 'grip0')->('gp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'wp') = fail
node: ('gp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('gp', 'gp', 'grip0')->('gp', 'gp', 'gp') = fail
node: ('wp

node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 2->12 (247.56/600.0 s, steps/err: 42(38.370847702 ms)/0.00181496755069)
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 2->13 (288.02/600.0 s, steps/err: 51(228.845119476 ms)/0.00190305665894)
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 2->14 (288.07/600.0 s, steps/err: 44(49.507856369 ms)/0.00220504404559)
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'gp', 'grip0'

node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = success
branching: 1->2 (15.79/600.0 s, steps/err: 55(585.020065308 ms)/0.00163044705602)
======================= terminated 0: first answer acquired ===============================
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = success
branching: 0->1 (0.05/600.0 s, steps/err: 25(31.38

node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = success
branching: 6->12 (22.3/600.0 s, steps/err: 49(64.3999576569 ms)/0.00173309083732)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
node: ('wp', 'gp', 'gp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = success
branching: 6->13 (22.39/600.0 s, steps/err: 54(73.6548900604 ms)/0.0014521295939)
node:

node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('gp', 'wp', 'gp')->('gp', 'grip0', 'gp') = fail
node: ('gp', 'wp', 'grip0')->('gp', 'wp', 'wp') = success
branching: 20->27 (44.43/600.0 s, steps/err: 51(87.0800018311 ms)/0.00114661147927)
======================= terminated 0: first answer acquired ===============================
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = f

In [23]:
for k,v in results_dict_3_shelf.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

0: [173.80621876716614, 2.4, 44.0, 34.4, 5, 10]
1: [102.03404826588101, 4.0, 50.44444444444444, 18.77777777777778, 1, 10]
2: [66.92647557788425, 6.444444444444445, 54.666666666666664, 12.11111111111111, 1, 10]
3: [26.401524199379814, 4.444444444444445, 34.77777777777778, 4.0, 1, 10]


In [24]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_shelf.json"), results_dict_3_shelf)

In [25]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210223-051658')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/filters/indy7/20210225-070527



### sampling 3-obj

In [26]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

results_dict_3_shelf = {0:[], 1:[], 2:[], 3:[]}
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}

In [27]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)


    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    mplan.update_gscene()
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)

    ppline.set_motion(mplan)
    ppline.set_sampler(tplan)

    gtimer = GlobalTimer.instance()
    gtimer.reset()



    for checker_idx in range(4):
        mplan.motion_filters = checkers_all[:checker_idx]
        goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
        mplan.reset_log(True)
        gtimer.tic("plan")
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
        elapsed = gtimer.toc("plan")/1000
        schedules = ppline.find_schedules()
        res = len(schedules)>0
        if res:
            schedule = ppline.sort_schedule(schedules)[0]
            move_num = len(schedule)-1
        else:
            move_num = 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_shelf[checker_idx].append([elapsed, move_num, plan_num, fail_num, res])

node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 0->1 (5.12/600.0 s, steps/err: 31(78.3789157867 ms)/0.000928614861359)
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp',

node: ('wp', 'gp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = success
branching: 1->17 (347.95/600.0 s, steps/err: 46(65.6769275665 ms)/0.00139947207325)
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'wp') = success
branching: 9->18 (368.19/600.0 s, steps/err: 46(81.5448760986 ms)/0.00161077205968)
node: ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('grip0', 'gp', 'wp')->('wp', 'gp', 'wp') = success
branching: 6->19 (383.3/600.0 s, steps/err: 8(19.9198722839 ms)/0.00182799876439)
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'gp', 'wp')->

node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->4 (25.41/600.0 s, steps/err: 29(28.8560390472 ms)/0.00102935953566)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->5 (30.48/600.0 s, steps/err: 31(26.4010429382 ms)/0.00141050626427)
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->6 (35.55/600.0 s, steps/err: 6(22.0699310303 ms)/0.00134187928969)
node: ('wp', 'wp', 'wp')->('wp'

node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->2 (0.09/600.0 s, steps/err: 29(32.6359272003 ms)/0.00145378138384)
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = success
branching: 1->3 (0.14/600.0 s, steps/err: 50(41.2669181824 ms)/0.00182640031477)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: (

node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = success
branching: 7->20 (24.33/600.0 s, steps/err: 32(417.53077507 ms)/0.00147173056555)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('grip0', 'wp', 'gp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'grip0', 'gp')->('wp', 'wp', 'gp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('grip0', 'wp', 'gp')->('gp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 6->21 (29.42/600.0 s, steps/err: 11(25.1159667969 ms)/0.00136834977364)
node: ('grip0', 'gp', 'wp')->('gp', 'gp', 'wp') = fail
node

...

node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'gp', 'gp')->('grip0', 'gp', 'gp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'gp') = fail
node: ('wp', 'gp', 'grip0')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'gp', 'gp')->('wp', 'grip0', 'gp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = success
branching: 24->36 (68.87/600.0 s, steps/err: 74(88.8059139252 ms)/0.00161646921331)
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'gp')->('grip0

node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = success
branching: 12->14 (6.75/600.0 s, steps/err: 51(65.6018257141 ms)/0.000964392285266)
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('grip0', 'gp', 'wp') = fail
node: ('wp', 'gp', 'wp')->('wp', 'gp', 'grip0') = success
branching: 11->15 (6.84/600.0 s, steps/err: 36(69.463968277 ms)/0.000884702850169)
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
no

node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 4->5 (277.1/600.0 s, steps/err: 67(90.0139808655 ms)/0.00157839233898)
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 4->6 (282.24/600.0 s, steps/err: 46(106.197834015 ms)/0.00157769364108)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 4->7 (287.31/600.0 s, steps/err: 31(37.3728275299 ms)/0.00170299679907)
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('

node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = success
branching: 0->3 (0.51/600.0 s, steps/err: 39(372.262001038 ms)/0.0012554750568)
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('grip0', 'wp', 'wp')->('gp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = success
branching: 0->4 (0.56/600.0 s, steps/err: 14(35.3178977966 ms)/0.000616055641399)
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'grip0', 'wp')->('wp', 'gp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = success
bran

node: ('wp', 'wp', 'wp')->('wp', 'grip0', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = fail
node: ('wp', 'wp', 'wp')->('wp', 'wp', 'grip0') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'gp') = fail
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->2 (5.17/600.0 s, steps/err: 37(54.7590255737 ms)/0.00169465651513)
node: ('wp', 'wp', 'wp')->('grip0', 'wp', 'wp') = success
branching: 0->3 (5.22/600.0 s, steps/err: 32(50.9090423584 ms)/0.000552696535301)
node: ('grip0', 'wp', 'wp')->('wp', 'wp', 'wp') = fail
node: ('wp', 'wp', 'grip0')->('wp', 'wp', 'wp') = success
branching: 1->4 (5.25/600.0 s, steps/err: 13(21.3851928711 ms)/0.000895628872983)
node: ('grip0', 'wp', 'wp')->('

In [28]:
for k,v in results_dict_3_shelf.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

0: [262.55660858154295, 2.8, 60.8, 52.0, 5, 10]
1: [132.76217827796935, 3.4, 65.6, 25.3, 0, 10]
2: [28.124668264389037, 3.2, 23.2, 5.1, 0, 10]
3: [18.60676951408386, 4.2, 34.7, 2.5, 0, 10]


In [29]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_shelf.json"), results_dict_3_shelf)

## sample with pole

In [20]:
Nmax_obj = 3

SAMPLE_NUM = 10

SHOW_PERIOD = 0.01

results_dict_3_pole = {0:[], 1:[], 2:[], 3:[]}
# results_dict_3_plane = {int(k): v for k, v in load_json(os.path.join(DATASET_PATH, "results_dict_3_plane.json")).items()}

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for i_s in range(SAMPLE_NUM):
    ## add floor, ceiling
    floor = Floor(gscene, "floor")
    gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                       dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                       color=floor.geometry.color, display=True, collision=True, fixed=True)
    # ceiling = Ceiling(gscene, "ceiling")

    ## set workplane
    wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
    pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane)

    gp_colliding = True
    while gp_colliding:
        ## set goalplane
        gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
        gp_colliding = wp.is_overlapped_with(gp.geometry)
    pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane)

    pole = Pole(gscene, "pole", RTH=[0.3, gp.RTH[1], 0])

    ## add object
    obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp)

    obj_set_list = []
    for i_obj, obj in enumerate(obj_list):
        obj_pscene, handles = add_object(pscene, obj)
        obj_set_list.append((obj, obj_pscene, handles))
    obj, obj_pscene, handles = obj_set_list[0]

    obj_pscene.geometry.color = (0.8,0.2,0.2,1)
    if VISUALIZE:
        gscene.set_rviz()

    mplan.update_gscene()
    initial_state = pscene.update_state(HOME_POSE)
    pscene.set_object_state(initial_state)
    from_state = initial_state.copy(pscene)

    ppline.set_motion(mplan)
    ppline.set_sampler(tplan)

    gtimer = GlobalTimer.instance()
    gtimer.reset()



    for checker_idx in range(1,4):
        mplan.motion_filters = checkers_all[:checker_idx]
        goal_nodes = [("gp",)+deepcopy(from_state.node)[1:]]
        mplan.reset_log(True)
        gtimer.tic("plan")
        ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=600, multiprocess=False, timeout=5)
        elapsed = gtimer.toc("plan")/1000
        schedules = ppline.find_schedules()
        res = len(schedules)>0
        if res:
            schedule = ppline.sort_schedule(schedules)[0]
            move_num = len(schedule)-1
        else:
            move_num = 0
        plan_num = len(mplan.result_log["planning"])
        fail_num = np.sum(np.logical_not(mplan.result_log["planning"]))
        results_dict_3_pole[checker_idx].append([elapsed, move_num, plan_num, fail_num, res])

In [22]:
for k,v in results_dict_3_pole.items():
    vv = np.array(v)
    if len(vv)>0:
        print("{}: {}".format(k, list(np.mean(vv[np.where(vv[:,-1])[0], :-1], axis=0))+[np.sum(np.logical_not(vv[:,-1])), len(vv)]))

1: [208.52604667345682, 4.666666666666667, 90.44444444444444, 40.0, 1, 10]
2: [137.5783672809601, 6.0, 100.1, 25.3, 0, 10]
3: [42.91876456472609, 4.222222222222222, 62.0, 6.555555555555555, 1, 10]


In [23]:
save_json(os.path.join(DATASET_PATH, "results_dict_3_pole.json"), results_dict_3_pole)